In [2]:
#1) Import all necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import ee
import geemap
import geopandas as gpd


In [3]:
import os
import sys

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)




In [4]:
# Import SHAPEFILE_PATH from constants
from constants import STUDY_BOUNDARY_PATH

# Import CHIRPS Daily: Climate Hazards Group InfraRed Precipitation With Station Data (Version 2.0 Final) and Study Boundary

https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_PENTAD

Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.

Funk, Chris, Pete Peterson, Martin Landsfeld, Diego Pedreros, James Verdin, Shraddhanand Shukla, Gregory Husak, James Rowland, Laura Harrison, Andrew Hoell & Joel Michaelsen. "The climate hazards infrared precipitation with stations-a new environmental record for monitoring extremes". Scientific Data 2, 150066. doi:10.1038/sdata.2015.66 2015.

In [5]:
#ee.Authenticate()
ee.Initialize()

In [19]:
start_date = '2001-01-01'
end_date = '2010-12-31'
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate(start_date, end_date)



In [20]:
precipitation.getInfo()


{'type': 'ImageCollection',
 'bands': [],
 'id': 'UCSB-CHG/CHIRPS/DAILY',
 'version': 1684531588834092,
 'properties': {'system:visualization_0_min': '1.0',
  'type_name': 'ImageCollection',
  'keywords': ['chg',
   'climate',
   'geophysical',
   'precipitation',
   'ucsb',
   'weather'],
  'thumb': 'https://mw1.google.com/ges/dd/images/CHIRPS_thumb.png',
  'description': '<p>Climate Hazards Group InfraRed Precipitation with Station data (CHIRPS)\nis a 30+ year quasi-global rainfall dataset. CHIRPS incorporates\n0.05° resolution satellite imagery with in-situ station data\nto create gridded rainfall time series for trend analysis and seasonal\ndrought monitoring.</p><p><b>Provider: <a href="https://chc.ucsb.edu/data/chirps">UCSB/CHG</a></b><br><p><b>Resolution</b><br>5566 meters\n</p><p><b>Cadence</b><br>\n  1 day\n</p><p><b>Bands</b><table class="eecat"><tr><th scope="col">Name</th><th scope="col">Description</th></tr><tr><td>precipitation</td><td><p>Precipitation</p></td></tr></tabl

In [21]:
# Sort the ImageCollection by date
sorted_collection = precipitation.sort('system:time_start')

# Get the first image
first_image = sorted_collection.first()

# Get the last image
last_image = sorted_collection.sort('system:time_start', False).first()

# Print the dates of the first and last images
print('First image date:', first_image.get('system:index').getInfo())
print('Last image date:', last_image.get('system:index').getInfo())


First image date: 20010101
Last image date: 20101230


In [22]:
# Calculate the average annual precipitation


# Ten years
num_years = 2010 - 2001 + 1
average_annual_precipitation = precipitation.sum().divide(num_years)

In [23]:

study_boundary = gpd.read_file(STUDY_BOUNDARY_PATH)


In [24]:
ee_boundary = geemap.geopandas_to_ee(study_boundary)



In [25]:
clipped_precipitation = average_annual_precipitation.clip(ee_boundary)


In [26]:
Map = geemap.Map()
Map.centerObject(ee_boundary, zoom=10)
Map.addLayer(clipped_precipitation, {'min': 0, 'max': 3000, 'palette': ['blue', 'green', 'yellow', 'red']}, 'Average Annual Precipitation')
Map


Map(center=[-21.999008038350066, -59.79579144156555], controls=(WidgetControl(options=['position', 'transparen…

In [27]:
num_bands = len(clipped_precipitation.bandNames().getInfo())
print(f"Number of bands: {num_bands}")
# Get the band names from the clipped raster
band_names = clipped_precipitation.bandNames().getInfo()

# Print the band names
print("Band names:")
for i, band in enumerate(band_names, 1):
    print(f"{i}. {band}")


Number of bands: 1
Band names:
1. precipitation


In [28]:
# Extract the first feature from the FeatureCollection
ee_boundary_feature = ee_boundary.first()

# Get the geometry of the feature
ee_boundary_geometry = ee_boundary_feature.geometry()

# Get the coordinates of the geometry
ee_boundary_coordinates = ee_boundary_geometry.coordinates().getInfo()

In [29]:
export_params = {
    'scale': 30, # Resolution in meters
    'region': ee_boundary_coordinates, # Export only the region of interest
    'crs': 'EPSG:4326', # Coordinate reference system (optional)
    'fileFormat': 'GeoTIFF', # Export format (GeoTIFF or other supported formats)
    'fileNamePrefix': 'average_annual_precipitation', # Prefix for the exported file name
}

export_task = ee.batch.Export.image.toDrive(
    image=clipped_precipitation,
    description='average_annual_precipitation',
    folder='precipitation_data',  # specify a folder in your Google Drive
    maxPixels=1e10,
    **export_params
)



In [30]:
# Start the export task
export_task.start()

A new empty folder will appear in the google drive associated with Earth Engine account.
The export process takes a considerable amount of time, progress can be monitored at
https://code.earthengine.google.com/tasks